## Sky Localization of Gravitational-Wave Event
Tutorial 2.6.1: skymap visualization/comparison and cross-matching galaxy catalog.

In this tutorial we will learn how:

1. to visualize and compare the sky localizations,

2. to cross-match a gravitational-wave sky localization with a galaxy catalog.

## Sky localization and interforometer network.
Here we show the importance in construction a network of interferometers for pinpointing the gravitational-wave source localization over the sky. The case of the GW190814 will be examined.

### Real time improving of the sky localizations of GW190814
GW190814 was first identified on 2019 August 14, 21:11:00 UTC as a loud two-detector event in LIGO Livingston and Virgo data (S/N 21.4 and 4.3). A [Notice](https://gcn.gsfc.nasa.gov/notices_l/S190814bv.lvc) was issued through NASA's Gamma-ray Coordinates Network 20 minutes later with a two-detector source localization computed using the rapid Bayesian algorithm BAYESTAR ([Singer & Price 2016](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.93.024013)).

Shortly thereafter, reanalyses including LIGO Hanford data were performed. A coincident gravitational-wave signal was identified in all three detectors. Results of these three-detector analyses were reported in the [GCN Circular 25324](https://gcn.gsfc.nasa.gov/gcn3/25324.gcn3) within 2.3 hr of the time of the event providing a three-detector localization [LIGO Scientific Collaboration & Virgo Collaboration 2019](https://iopscience.iop.org/article/10.3847/2041-8213/ab960f).

We will apply the method [`ligo_skymap_contour_moc`](https://lscsoft.docs.ligo.org/ligo.skymap/tool/ligo_skymap_contour_moc.html) from [`ligo.skymap`](https://lscsoft.docs.ligo.org/ligo.skymap/) to create the 90% credible regions. 

In [1]:
# Download the two-detector sky maps of GW190814 from GraceDB.
!curl -O https://gracedb.ligo.org/api/superevents/S190814bv/files/bayestar.multiorder.fits,0
    
# Define the 90% credible regions of two-detector source localization.
!ligo-skymap-contour-moc bayestar.multiorder.fits,0  -c 90  --output 'GW190814 with 2 IFO'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  759k  100  759k    0     0   288k      0  0:00:02  0:00:02 --:--:--  288k
2021-04-06 17:24:04,030 WARNING Matplotlib created a temporary config/cache directory at /var/folders/5n/cq85zsfj29q5_nvkjg9l0b4c0000gn/T/matplotlib-mnsmu5dw because the default path (/Users/gius/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
2021-04-06 17:24:04,094 INFO Generating new fontManager, this may take some time...
2021-04-06 17:24:04,308 INFO Failed to extract font properties from /System/Library/Fonts/LastResort.otf: tuple indices must be integers or slices, not str
2021-04-06 17:24:04,319 INFO Failed to extract font properties from /System/Library/Fonts/Apple Colo

In [2]:
# Download the three-detector sky maps of GW190814 from GraceDB.
!curl -O https://gracedb.ligo.org/api/superevents/S190814bv/files/bayestar.multiorder.fits
    
# Define the 90% credible regions of three-detector source localization.
!ligo-skymap-contour-moc bayestar.multiorder.fits  -c 90  --output 'GW190814 with 3 IFO'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  759k  100  759k    0     0   268k      0  0:00:02  0:00:02 --:--:--  268k
2021-04-06 17:24:10,109 WARNING Matplotlib created a temporary config/cache directory at /var/folders/5n/cq85zsfj29q5_nvkjg9l0b4c0000gn/T/matplotlib-mc9uefx2 because the default path (/Users/gius/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
2021-04-06 17:24:10,173 INFO Generating new fontManager, this may take some time...
2021-04-06 17:24:10,370 INFO Failed to extract font properties from /System/Library/Fonts/LastResort.otf: tuple indices must be integers or slices, not str
2021-04-06 17:24:10,414 INFO Failed to extract font properties from /System/Library/Fonts/Supplement

In [3]:
# Initialize an Aladin widget at a defined sky's position and with the DSS color survey.
import ipyaladin as ipyal

aladin = ipyal.Aladin(target='16.91000 -28.1100', fov=180, survey='P/DSS2/color')
aladin

Aladin(fov=180.0, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', '…

In [4]:
# Show the credible regions in the Aladin widget with 2 different colors.
colors = ["magenta", "yellow"]

credible_regions = ["GW190814 with 2 IFO", "GW190814 with 3 IFO"]

for credible_region, color in zip(credible_regions, colors):
    aladin.add_moc_from_URL(credible_region, {'color': color, 'opacity': 0.7,
                            'adaptativeDisplay': False, 'name':credible_region})

Scroll up and visualize the credible region contours in the interactive Aladin widget. From the `Manage Layers`, <img src="images/ipyaladin_layer.png" alt="the Layer Button" style="width:30px; display: inline-block;"/> the 2 credible regions can be independently selected. 

If you hover the mouse pointer over the credible region plan a message will be displayed above it to show the coverage in sky percentage. How many square degrees? How refined the localization is?

## Cross-Matching the sky localization with a Galaxy Catalog
All sky localization for CBC events are three dimensional: they include both the sky probability map and a directionally dependent distance estimate. This can be useful for identifying possible host galaxies using a galaxy redshift catalog.

Using the [`crossmatch`](https://lscsoft.docs.ligo.org/ligo.skymap/postprocess/crossmatch.html) method in [`ligo.skymap`](https://lscsoft.docs.ligo.org/ligo.skymap/) package, we can extract the galaxies within the 90% credible volume. The galaxy table will be showed in the previous Aladin widget as a new overlay layer.

In [5]:
from astroquery.vizier import VizierClass
from astropy.coordinates import SkyCoord
from ligo.skymap.io import read_sky_map
from ligo.skymap.postprocess import crossmatch

Matplotlib created a temporary config/cache directory at /var/folders/5n/cq85zsfj29q5_nvkjg9l0b4c0000gn/T/matplotlib-daktfgqf because the default path (/Users/gius/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Now we retrieve the GLADE catalog.

**NOTE**. To keep the MyBinder memory, we load a verified subsample from the GLADE catalog.
For real applications, download the entire [GLADE](https://vizier.u-strasbg.fr/viz-bin/VizieR?-source=VII/281) catalog.

In [6]:
# For real application, uncomment the lines below to load the entire GLADE catalog.

#vizier = VizierClass(
#    row_limit=-1, columns=['GWGC', '_RAJ2000', '_DEJ2000', 'Dist'])
#cat, = vizier.get_catalogs('VII/281/glade2')

In [7]:
# Comment this line in real application. We use this to save MyBinder memory.
from astropy.table import Table
cat = Table.read('https://github.com/ggreco77/Tutotest/blob/main/resources/table_tuto?raw=true')

In [8]:
# Get coordinates and distance columns.
coordinates = SkyCoord(cat['_RAJ2000'], cat['_DEJ2000'], cat['Dist'])

In [9]:
# Read the initial GW190814 skymap with 3 IFO.
skymap = read_sky_map('bayestar.multiorder.fits', moc=True)

# List the galaxies within the 90% credible volume.
result = crossmatch(skymap, coordinates)
table = cat[result.searched_prob_vol < 0.9]

In [10]:
# Add the galaxy list in the previous Aladin widget.
aladin.add_table(table)